In [4]:
from flask import Flask, render_template,jsonify,request, url_for
import pandas as pd
import plotly.express as px
import os
import random

app = Flask(__name__)

df1 = pd.DataFrame({
    'Nombre': ['Reservas0', 'Reservas1', 'Reservas2'],
    'Descripción': ["LOREM IPSUM", "LOREM IPSUM", "LOREM IPSUM"],
    'Staff': [3, 5, 2],
    'Senior': [3, 3, 0],
    'Manager': [1, 1, 1],
    'Senior Manager': [1,0, 0],
    'Socio': [0, 1, 0]
})

df12 = pd.DataFrame({
    'Nombre': ['Reservas0', 'Reservas1', 'Reservas2'],
    'Descripción': ["LOREM IPSUM", "LOREM IPSUM", "LOREM IPSUM"],
    'Staff': [3, 5, 2],
    'Senior': [3, 3, 0],
    'Manager': [1, 1, 1],
    'Senior Manager': [1,0, 0],
    'Socio': [0, 1, 0]
})

df13 = pd.DataFrame({
    'Nombre': ['Reservas0', 'Reservas1', 'Reservas2'],
    'Descripción': ["LOREM IPSUM", "LOREM IPSUM", "LOREM IPSUM"],
    'Staff': [3, 5, 2],
    'Senior': [3, 3, 0],
    'Manager': [1, 1, 1],
    'Senior Manager': [1,0, 0],
    'Socio': [0, 1, 0]
})

df2 = pd.DataFrame({
    'Staff': [1],
    'Senior': [3],
    'Manager': [5],
    'Senior Manager': [10],
    'Socio': [15]
})

df22 = pd.DataFrame({
    'Staff': [1],
    'Senior': [3],
    'Manager': [5],
    'Senior Manager': [10],
    'Socio': [15]
})

df23 = pd.DataFrame({
    'Staff': [1],
    'Senior': [3],
    'Manager': [5],
    'Senior Manager': [10],
    'Socio': [15]
})

df3 = pd.DataFrame({
    'Costo': [350]
})

df32 = pd.DataFrame({
    'Costo': [350]
})

df33 = pd.DataFrame({
    'Costo': [350]
})

df4 = pd.concat([df1[['Nombre','Descripción']],(df1[['Staff','Senior','Manager','Senior Manager','Socio']].multiply(df2.values))*int(df3.loc[0])],axis=1)

df42 = pd.concat([df12[['Nombre','Descripción']],(df12[['Staff','Senior','Manager','Senior Manager','Socio']].multiply(df22.values))*int(df32.loc[0])],axis=1)

df43 = pd.concat([df13[['Nombre','Descripción']],(df13[['Staff','Senior','Manager','Senior Manager','Socio']].multiply(df23.values))*int(df33.loc[0])],axis=1)

dataframes = [df1,df2,df3,df4]
titles = ['Servicios','Rangos','Costo staff','Presupuesto']

dataframes2 = [df12,df22,df32,df42]
titles2 = ['Servicios','Rangos','Costo staff','Presupuesto']

dataframes3 = [df13,df23,df33,df43]
titles3 = ['Servicios','Rangos','Costo staff','Presupuesto']

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', title='Inicio')

@app.route('/proyectos')
def proyectos():
    return render_template('proyectos.html')

@app.route('/tables1')
def tables1():
    return render_template('tables1.html', dataframes=dataframes,titles=titles)

@app.route('/tables2')
def tables2():
    return render_template('tables2.html', dataframes2=dataframes2,titles=titles)

@app.route('/tables3')
def tables3():
    return render_template('tables3.html', dataframes3=dataframes3,titles=titles)

@app.route('/update_cell', methods=['POST'])
def update_dataframe():
    data = request.get_json()
    row_index = int(data['row_index'])
    column_name = data['column_index']
    new_value = data['value']
    df_index = int(data['table_index'])
    df = dataframes[df_index]
    if row_index >= df.shape[0]:
        new_row = pd.Series({'Nombre': '', 'Descripción': '', 'Staff': 0, 'Senior': 0, 'Manager': 0, 'Senior Manager': 0, 'Socio': 0})
        df = df.append(new_row, ignore_index=True)
    df.iloc[row_index, column_name] = int(new_value)
    dataframes[df_index] = df
    response_data = df.to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/update_cell2', methods=['POST'])
def update_dataframe2():
    data = request.get_json()
    row_index = int(data['row_index'])
    column_name = data['column_index']
    new_value = data['value']
    df2_index = int(data['table_index'])
    df2 = dataframes2[df2_index]
    if row_index >= df2.shape[0]:
        new_row = pd.Series({'Nombre': '', 'Descripción': '', 'Staff': 0, 'Senior': 0, 'Manager': 0, 'Senior Manager': 0, 'Socio': 0})
        df2 = df2.append(new_row, ignore_index=True)
    df2.iloc[row_index, column_name] = int(new_value)
    dataframes2[df2_index] = df2
    response_data = df2.to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/update_cell3', methods=['POST'])
def update_dataframe3():
    data = request.get_json()
    row_index = int(data['row_index'])
    column_name = data['column_index']
    new_value = data['value']
    df3_index = int(data['table_index'])
    df3 = dataframes3[df3_index]
    if row_index >= df3.shape[0]:
        new_row = pd.Series({'Nombre': '', 'Descripción': '', 'Staff': 0, 'Senior': 0, 'Manager': 0, 'Senior Manager': 0, 'Socio': 0})
        df3 = df3.append(new_row, ignore_index=True)
    df3.iloc[row_index, column_name] = int(new_value)
    dataframes[df3_index] = df3
    response_data = df3.to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/recalculate_df4', methods=['POST'])
def recalculate_df4():
    global dataframes
    dataframes[3] = pd.concat([dataframes[0][['Nombre','Descripción']],(dataframes[0][['Staff','Senior','Manager','Senior Manager','Socio']].multiply(dataframes[1].values))*int(dataframes[2].loc[0])],axis=1)
    response_data = dataframes[3].to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/recalculate_df42', methods=['POST'])
def recalculate_df42():
    global dataframes2
    dataframes2[3] = pd.concat([dataframes2[0][['Nombre','Descripción']],(dataframes2[0][['Staff','Senior','Manager','Senior Manager','Socio']].multiply(dataframes2[1].values))*int(dataframes2[2].loc[0])],axis=1)
    response_data = dataframes2[3].to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/recalculate_df43', methods=['POST'])
def recalculate_df43():
    global dataframes3
    dataframes3[3] = pd.concat([dataframes3[0][['Nombre','Descripción']],(dataframes3[0][['Staff','Senior','Manager','Senior Manager','Socio']].multiply(dataframes3[1].values))*int(dataframes3[2].loc[0])],axis=1)
    response_data = dataframes3[3].to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/update_df1', methods=['POST'])
def update_df1():
    data = request.get_json()
    data_dict = {
    'Nombre': [str(data[-1][0])],
    'Descripción': [str(data[-1][1])],
    'Staff': [int(data[-1][2])],
    'Senior': [int(data[-1][3])],
    'Manager': [int(data[-1][4])],
    'Senior Manager': [int(data[-1][5])],
    'Socio': [int(data[-1][6])]
    }
    new_row = pd.DataFrame(data_dict)
    dataframes[3] = pd.concat([dataframes[3], new_row], axis=0)
    dataframes[3] = dataframes[3].reset_index(drop=True)
    dataframes[0] = pd.concat([dataframes[0], new_row], axis=0)
    response_data = dataframes[3].to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/update_df12', methods=['POST'])
def update_df12():
    data = request.get_json()
    data_dict = {
    'Nombre': [str(data[-1][0])],
    'Descripción': [str(data[-1][1])],
    'Staff': [int(data[-1][2])],
    'Senior': [int(data[-1][3])],
    'Manager': [int(data[-1][4])],
    'Senior Manager': [int(data[-1][5])],
    'Socio': [int(data[-1][6])]
    }
    new_row = pd.DataFrame2(data_dict)
    dataframes2[3] = pd.concat([dataframes2[3], new_row], axis=0)
    dataframes2[3] = dataframes2[3].reset_index(drop=True)
    dataframes2[0] = pd.concat([dataframes2[0], new_row], axis=0)
    response_data = dataframes2[3].to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

@app.route('/update_df13', methods=['POST'])
def update_df13():
    data = request.get_json()
    data_dict = {
    'Nombre': [str(data[-1][0])],
    'Descripción': [str(data[-1][1])],
    'Staff': [int(data[-1][2])],
    'Senior': [int(data[-1][3])],
    'Manager': [int(data[-1][4])],
    'Senior Manager': [int(data[-1][5])],
    'Socio': [int(data[-1][6])]
    }
    new_row = pd.DataFrame3(data_dict)
    dataframes3[3] = pd.concat([dataframes3[3], new_row], axis=0)
    dataframes3[3] = dataframes3[3].reset_index(drop=True)
    dataframes3[0] = pd.concat([dataframes3[0], new_row], axis=0)
    response_data = dataframes3[3].to_json(orient='records')
    return jsonify({'status': 'success', 'data': response_data})

litigios = []

bufetes = [
    { 'nombre': 'LexGlobal Consultores', 'ubicacion': 'Puebla', 'juiciosAsignados': 0 },
    { 'nombre': 'Justicia y Aliados', 'ubicacion': 'México', 'juiciosAsignados': 0 },
    { 'nombre': 'Soluciones Legales Integrales', 'ubicacion': 'Mérida', 'juiciosAsignados': 0 },
    { 'nombre': 'Abogados Unidos', 'ubicacion': 'Puebla', 'juiciosAsignados': 0 },
    { 'nombre': 'Firma Jurídica Excelencia', 'ubicacion': 'México', 'juiciosAsignados': 0 },
    { 'nombre': 'Defensores Legales Puebla', 'ubicacion': 'Puebla', 'juiciosAsignados': 0 }
]

@app.route('/ruleta', methods=['POST','GET'])
def ruleta():
    bufete_seleccionado = None
    litigios_por_bufete = pd.DataFrame()
    acumulado_por_bufete = pd.DataFrame()
    if request.method == 'POST':
        location = request.form['location']
        litigio_id = request.form['litigio_id']
        bufetes_filtrados = [bufete for bufete in bufetes if bufete['ubicacion'] == location]
        min_assigned_litigios = min([bufete['juiciosAsignados'] for bufete in bufetes_filtrados])
        bufetes_min_assigned = [bufete for bufete in bufetes_filtrados if bufete['juiciosAsignados'] == min_assigned_litigios]
        bufete_seleccionado = random.choice(bufetes_min_assigned)
        litigios.append({
            'id': litigio_id,
            'estado': location,
            'bufete': bufete_seleccionado['nombre']
        })
        bufete_seleccionado['juiciosAsignados'] += 1
        litigios_por_bufete = pd.DataFrame(litigios).groupby(['bufete', 'estado']).agg({'id': 'count'}).rename(columns={'id': 'total_juicios_asignados'})
        acumulado_por_bufete = pd.DataFrame([{'bufete': bufete['nombre'], 'total_juicios_asignados': bufete['juiciosAsignados']} for bufete in bufetes])
    return render_template('litigio.html', bufete=bufete_seleccionado, litigios_por_bufete=litigios_por_bufete.to_dict(), acumulado_por_bufete=acumulado_por_bufete.to_dict())

@app.route('/select_bufete', methods=['GET'])
def select_bufete():
    location = request.args.get('location')
    bufetes_filtrados = [bufete for bufete in bufetes if bufete['ubicacion'] == location]
    min_assigned_litigios = min([bufete['juiciosAsignados'] for bufete in bufetes_filtrados])
    bufetes_min_assigned = [bufete for bufete in bufetes_filtrados if bufete['juiciosAsignados'] == min_assigned_litigios]
    bufete_seleccionado = random.choice(bufetes_min_assigned)
    bufete_seleccionado['juiciosAsignados'] += 1
    print(bufete_seleccionado)
    return jsonify(bufete_seleccionado)

sucursales_data = {
  'Mexico': {
    'Justicia y Aliados': ['9:00am', '10:00am', '11:00am'],
    'Firma Juridica Excelencia': ['10:00am', '11:00am', '12:00pm']
  },
  'Puebla': {
    'LexGlobal Consultores': ['9:00am', '10:00am', '11:00am'],
    'Abogados Unidos': ['10:00am', '11:00am', '12:00pm'],
    'Defensores Legales Puebla': ['10:00am', '11:00am', '12:00pm']
  },
  'Merida': {
    'Soluciones Legales Integrales': ['9:00am', '10:00am', '11:00am']
  }
}

@app.route('/sucursales/<ubicacion>')
def sucursales(ubicacion):
    if ubicacion in sucursales_data:
        sucursales = list(sucursales_data[ubicacion].keys())
        url_horarios = url_for('horarios', ubicacion=ubicacion, sucursal=sucursales[0])
        return jsonify({'sucursales': sucursales, 'url_horarios': url_horarios})
    else:
        return jsonify({'sucursales': []})

@app.route('/horarios/<ubicacion>/<sucursal>')
def horarios(ubicacion, sucursal):
    if ubicacion in sucursales_data and sucursal in sucursales_data[ubicacion]:
        horarios = sucursales_data[ubicacion][sucursal]
        return jsonify({'horarios': horarios})
    else:
        return jsonify({'horarios': []})

@app.route('/filtros')
def filtros():
    return render_template('filtros.html')

@app.route('/litigios', methods=['GET'])
def get_litigios():
    return jsonify(litigios)

@app.route('/litigios', methods=['POST'])
def add_litigio():
    data = request.get_json()
    litigios.append(data)
    return jsonify(litigios)

@app.route('/litigios/<int:litigio_id>/asignar', methods=['PUT'])
def asignar_litigio(litigio_id):
    data = request.get_json()
    estado = data['estado']
    bufete = data['bufete']
    for litigio in litigios:
        if litigio['id'] == litigio_id:
            litigio['estado'] = estado
            litigio['bufete'] = bufete
            for b in bufetes:
                if b['nombre'] == bufete:
                    b['juiciosAsignados'] += 1
            return jsonify(litigio)
    return jsonify({'message': 'Litigio no encontrado'})

@app.route('/bufetes', methods=['GET'])
def get_bufetes():
    return jsonify(bufetes)

@app.route('/bufetes/<bufete_nombre>/acumulado', methods=['GET'])
def get_acumulado(bufete_nombre):
    total_juicios = 0
    for b in bufetes:
        if b['nombre'] == bufete_nombre:
            total_juicios = b['juiciosAsignados']
    return jsonify({'bufete': bufete_nombre, 'totalJuicios': total_juicios})

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/May/2023 19:07:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2023 19:07:18] "GET /ruleta HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2023 19:07:21] "GET /select_bufete?location=Puebla HTTP/1.1" 200 -


{'nombre': 'LexGlobal Consultores', 'ubicacion': 'Puebla', 'juiciosAsignados': 1}


127.0.0.1 - - [18/May/2023 19:07:26] "GET /select_bufete?location=Puebla HTTP/1.1" 200 -


{'nombre': 'Abogados Unidos', 'ubicacion': 'Puebla', 'juiciosAsignados': 1}


...

{'nombre': 'LexGlobal Consultores', 'ubicacion': 'Puebla', 'juiciosAsignados': 4}


127.0.0.1 - - [18/May/2023 19:08:34] "GET /select_bufete?location=Puebla HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2023 19:08:34] "GET /select_bufete?location=Puebla HTTP/1.1" 200 -


{'nombre': 'LexGlobal Consultores', 'ubicacion': 'Puebla', 'juiciosAsignados': 5}
{'nombre': 'Defensores Legales Puebla', 'ubicacion': 'Puebla', 'juiciosAsignados': 5}


127.0.0.1 - - [18/May/2023 19:08:34] "GET /select_bufete?location=Puebla HTTP/1.1" 200 -


{'nombre': 'Abogados Unidos', 'ubicacion': 'Puebla', 'juiciosAsignados': 5}
